In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/train_germination.npy')
val_x = np.load('/home/ckwwk/data/germination_data/val_germination.npy')
test_x = np.load('/home/ckwwk/data/germination_data/test_germination.npy')

In [4]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/label/germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [5]:
from keras.utils import to_categorical
train_y = label_df[0:1000]
train_y = to_categorical(train_y)

In [6]:
val_y = label_df[1000:1250]
val_y = to_categorical(val_y)
test_y = label_df[1250:1536]
test_y = to_categorical(test_y)

In [7]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [8]:
input_layer = Input((61, 61, 275, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu',padding='same')(input_layer)
conv_layer2 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer1)

## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(1, 1, 3))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu',padding='same')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer4)

conv_layer5 = Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu',padding='same')(pooling_layer2)
conv_layer6 = Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer5)
pooling_layer3 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer6)

conv_layer7 = Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu',padding='same')(pooling_layer3)
conv_layer8 = Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer7)
pooling_layer4 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer8)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer5 = BatchNormalization()(pooling_layer4)
flatten_layer = Flatten()(pooling_layer5)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

W0806 14:11:21.550573 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 14:11:21.552160 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 14:11:21.557935 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 14:11:21.751928 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pleas

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 275, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 61, 61, 275, 16)   448       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 61, 61, 275, 32)   13856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 61, 61, 91, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 61, 61, 91, 32)    27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 61, 61, 91, 64)    55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 30, 30, 30, 64)    0         
__________

In [10]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model,gpus=4)

W0806 14:11:21.866433 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [11]:
adam = Adam(lr=0.01, decay=1e-05)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W0806 14:11:23.494960 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
filepath = "best-model_V3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='/home/ckwwk/tensor_log_V3',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [13]:
history = model.fit(x=train_x, y=train_y, batch_size=12, epochs=500, validation_data=(val_x,val_y),callbacks=callbacks_list)

W0806 14:11:30.148027 140340058576704 deprecation.py:323] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 250 samples


W0806 14:11:39.474955 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0806 14:11:39.516039 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0806 14:11:39.518877 140340058576704 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ckwwk/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/500
1000/1000 [==============================] - 98s 98ms/step - loss: 5.8595 - acc: 0.6290 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00001: acc improved from -inf to 0.62900, saving model to best-model_V3.hdf5
Epoch 2/500
1000/1000 [==============================] - 80s 80ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00002: acc improved from 0.62900 to 0.63200, saving model to best-model_V3.hdf5
Epoch 3/500
1000/1000 [==============================] - 78s 78ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00003: acc did not improve from 0.63200
Epoch 4/500
1000/1000 [==============================] - 78s 78ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 00004: acc improved from 0.63200 to 0.63200, saving model to best-model_V3.hdf5
Epoch 5/500
1000/1000 [==============================] - 78s 78ms/step - loss: 5.9315 - acc: 0.6320 - val_loss: 6.1893 - val_acc: 0.6160

Epoch 000

KeyboardInterrupt: 

In [ ]:
# load best weights
model.load_weights("best-model_V3.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
#model.predict(test_x,batch_size=10)
model.evaluate(test_x,test_y,batch_size=32)